In [3]:
import astropy.units as u
import datetime
import math
from astropy.coordinates import AltAz, EarthLocation, SkyCoord, ICRS
from astropy.time import Time
from ugradio import nch, doppler

# RPI TIME MAY BE WRONG

# lab location
lab = EarthLocation(lat=nch.lat*u.deg, lon=nch.lon*u.deg, height=nch.alt*u.m)

Error loading librtlsdr. Make sure librtlsdr (and all of its dependencies) are in your path


In [4]:
def get_julian_datetime(date):
    """
    Convert a datetime object into julian float.
    Args:
        date: datetime-object of date in question

    Returns: float - Julian calculated datetime.
    Raises: 
        TypeError : Incorrect parameter type
        ValueError: Date out of range of equation
    """

    # Ensure correct format
    if not isinstance(date, datetime.datetime):
        raise TypeError('Invalid type for parameter "date" - expecting datetime')
    elif date.year < 1801 or date.year > 2099:
        raise ValueError('Datetime must be between year 1801 and 2099')

    # Perform the calculation
    julian_datetime = 367 * date.year - int((7 * (date.year + int((date.month + 9) / 12.0))) / 4.0) + int(
        (275 * date.month) / 9.0) + date.day + 1721013.5 + (
                          date.hour + date.minute / 60.0 + date.second / math.pow(60,
                                                                                  2)) / 24.0 - 0.5 * math.copysign(
        1, 100 * date.year + date.month - 190002.5) + 0.5

    return julian_datetime

# calculate galactic(120, 0) at current time

In [8]:
# target coordinates
c = SkyCoord(frame="galactic", l=120*u.deg, b=0*u.deg)

# observation time
obstime = Time(datetime.datetime.utcnow(), scale='utc', location=lab)

# output coordinates
c.transform_to(AltAz(obstime=obstime, location=lab))

<SkyCoord (AltAz: obstime=2023-02-24 04:08:58.919839, location=(-2690570.65408925, -4263131.81441035, 3894417.3498527) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (326.4789391, 34.40670492)>

# calculate galactic(120, 0) at some given time

In [9]:
# target coordinates
c = SkyCoord(frame="galactic", l=120*u.deg, b=0*u.deg, obstime=obstime)

# observation time
obstime = Time(datetime.datetime(2023, 2, 17, 3, 35), scale='utc', location=lab)

# output coordinates
c.transform_to(AltAz(obstime=obstime, location=lab))

<SkyCoord (AltAz: obstime=2023-02-17 03:35:00, location=(-2690570.65408925, -4263131.81441035, 3894417.3498527) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (324.81378774, 41.30090627)>

# LSR correction 
based on feb 18 data

In [10]:
# target coordinates
c = SkyCoord(frame="galactic", l=120*u.deg, b=0*u.deg, obstime=obstime)

# observation time
obs_datetime = datetime.datetime(2023, 2, 18, 23, 29)
obstime = Time(obs_datetime, scale='utc', location=lab)

# output coordinates
c.transform_to(ICRS())

<SkyCoord (ICRS): (ra, dec) in deg
    (6.45083114, 62.72572675)>

In [11]:
doppler.get_projected_velocity(6.45083114, 62.72572675, get_julian_datetime(obs_datetime))

<Quantity -17456.72744228 m / s>